# COVID-19 Predictions with ARIMA model

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Recovered']
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Diamond Princess']
confirmed_cases_global.head()


,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653
1,NaN,Albania,0,0,0,0,0,0,0,0,...,868,872,876,880,898,916,933,946,948,949
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,755,755,758,760,761,761,761,761,761,761
4,NaN,Angola,0,0,0,0,0,0,0,0,...,45,45,45,45,48,48,48,48,50,52


In [3]:
#create a row for Canada
confirmed_cases_Canada=confirmed_cases_global[confirmed_cases_global['Country/Region']=='Canada']
confirmed_cases_Canada=confirmed_cases_Canada.groupby(['Country/Region']).sum().reset_index()
confirmed_cases_Canada.insert(0, 'Province/State', float('NaN'))
confirmed_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,NaN,Canada,0,0,0,0,1,1,2,2,...,70090,71263,72418,73567,74780,75958,77205,78331,79410,80492


In [4]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,151,151,152,152,152,154,154,154,154,154
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,16,19,19,19,19,19,21,21,21,21
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,2198,2256,2299,2329,2427,2542,2589,2646,2710,2805
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,69,69,69,69,69,69,69,69,69,69


In [5]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [6]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Recovered']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Diamond Princess']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,120,122,127,132,136,153,168,169,173,178
1,NaN,Albania,0,0,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,502,507,515,522,529,536,542,548,555,561
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,48,48,48,49,49,49,51,51,51,51
4,NaN,Angola,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,3,3


In [7]:
deaths_cases_Canada=deaths_cases_global[deaths_cases_global['Country/Region']=='Canada']
deaths_cases_Canada=deaths_cases_Canada.groupby(['Country/Region']).sum().reset_index()
deaths_cases_Canada.insert(0, 'Province/State', float('NaN'))
deaths_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,NaN,Canada,0,0,0,0,0,0,0,0,...,4990,5114,5299,5424,5591,5678,5799,5902,5959,6027


In [8]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,111,113,114,115,117,122,122,123,124,124
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,4,5,6,6,6,6,6,6,6,6


In [9]:
deaths_cases_US = deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()


In [10]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US,confirmed_cases_Canada], axis=0, sort=False)
confirmed_cases.to_csv('confirmed_cases_total.csv', sep=',',index = False)

In [11]:
deaths_cases = pd.concat([deaths_cases_global, deaths_cases_US,deaths_cases_Canada], axis=0, sort=False)
deaths_cases.to_csv('deaths_cases_total.csv', sep=',',index = False)

In [12]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()
recoveries_cases.to_csv('recoveries_cases_total.csv', sep=',',index = False)

In [13]:
colnames = confirmed_cases.columns.values

In [14]:
colonames = recoveries_cases.columns.values

In [15]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [16]:
time_date = confirmed.columns.values

In [17]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [18]:
pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [19]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [20]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Tajikistan','Comoros'])]

#confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'].isin(['Tajikistan'])]
confirmed_cases

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653
1,NaN,Albania,0,0,0,0,0,0,0,0,...,868,872,876,880,898,916,933,946,948,949
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,755,755,758,760,761,761,761,761,761,761
4,NaN,Angola,0,0,0,0,0,0,0,0,...,45,45,45,45,48,48,48,48,50,52
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,25,25,25,25,25,25,25,25,25,25
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,6034,6278,6563,6879,7134,7479,7805,8068,8371,8809
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,3313,3392,3538,3718,3860,4044,4283,4472,4823,5041
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,107,107,107,107,107,107,107,107,107,107
9,New South Wales,Australia,0,0,0,0,3,4,4,4,...,3053,3053,3059,3063,3071,3074,3075,3076,3078,3081


In [21]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [22]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}

In [23]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [24]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [25]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [26]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()
    
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [27]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
            
    #use the arima model to make predictions, use original timeseries with optimal d:
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    #print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    
    return forecasts , conf_int  # returns a tuple, array of forecasts and all conf int

In [28]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    df = df.set_index('date')
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead) #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [29]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation, pred_days_ahead):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead = 0  
        region = region
        best_pred = 0
        best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead, region, best_pred, best_conf_int, reject_country)
     
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
   
    best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    if (best_conf_int[0]>best_conf_int[1]):
        best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead<reg[-1][-1]):
        best_pred_ahead= reg[-1][-1]+pred_days_ahead
    
    reject_country = 0
    
    return best_pred_ahead, region, best_pred, best_conf_int, reject_country

In [30]:
pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
pred_file

'2day_prediction_2020-05-20_arima.csv'

In [31]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions = pd.DataFrame(columns = columns)
    
    k = k + 1
       
    best_pred_confirmed, region_con, best_pred_con, conf_int_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths, region_d, best_pred_d, conf_int_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries, region_r, best_pred_r, conf_int_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value=int(best_pred_recoveries) 
        low_R=int(conf_int_r[0])
        high_R=int(conf_int_r[1])
    else:
        R_value=''
        low_R=''
        high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical=serious_percentage*best_pred_confirmed
    low_critical=serious_percentage*conf_int_con[0]
    high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': R_value ,
                                                'low95R' : low_R,
                                                'high95R' : high_R,
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': int(critical) ,
                                                'low95C':  int(low_critical) ,
                                                'high95C': int(high_critical) },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/10/20  5/11/20  5/12/20  5/13/20  \
0        0        0        0  ...     4402     4687     4963     5226   

   5/14/20  5/15/20  5/16/20  5/17/20  5/18/20  5/19/20  
0     5639     6053     6402     6664     7072     7653  

[1 rows x 121 columns]
Fit ARIMA: (0, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=968.949, BIC=973.787, Time=0.080 seconds
Fit ARIMA: (0, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=899.876, BIC=907.132, Time=0.163 seconds
Fit ARIMA: (0, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=865.503, BIC=875.178, Time=0.200 seconds
Fit ARIMA: (0, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=867.049, BIC=879.143, Time=0.306 seconds
Fit ARIMA: (0, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=859.236, BIC=873.749, Time=0.620 seconds
Fit ARIMA: (1, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=935.681, BIC=942.

Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=477.641, BIC=493.480, Time=0.265 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=471.741, BIC=480.792, Time=0.124 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=473.722, BIC=485.036, Time=0.227 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=475.302, BIC=488.878, Time=0.793 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=477.230, BIC=493.069, Time=0.533 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=473.725, BIC=485.039, Time=0.184 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=475.727, BIC=489.303, Time=0.223 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=477.258, BIC=493.097, Time=0.440 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=475.715, BIC=489.291, Time=0.301 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=477.713, BIC=493.552, Time=0.291 seconds
Total fit time: 4.639 seconds
  Province

Fit ARIMA: (0, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=394.544, BIC=401.113, Time=0.208 seconds
Fit ARIMA: (0, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=383.845, BIC=392.603, Time=0.220 seconds
Fit ARIMA: (0, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=383.294, BIC=394.242, Time=0.296 seconds
Fit ARIMA: (0, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=384.993, BIC=398.131, Time=0.455 seconds
Fit ARIMA: (1, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=418.238, BIC=424.807, Time=0.031 seconds
Fit ARIMA: (1, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=382.740, BIC=391.498, Time=0.234 seconds
Fit ARIMA: (1, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=384.243, BIC=395.191, Time=0.384 seconds
Fit ARIMA: (1, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=385.124, BIC=398.262, Time=0.239 seconds
Fit ARIMA: (1, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=385.258, BIC=400.586, Time=0.498 seconds
Fit ARIMA: (2, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=401.360, BIC=410.119, Time=0.078 seconds
Fit ARIMA: (2, 3, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=171.958, BIC=184.217, Time=0.386 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=173.123, BIC=187.425, Time=0.463 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=175.673, BIC=187.931, Time=0.062 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=173.397, BIC=187.698, Time=0.483 seconds
Total fit time: 5.369 seconds
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
3            NaN        Andorra        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  5/10/20  5/11/20  5/12/20  5/13/20  \
3        0        0        0  ...      550      550      568      576   

   5/14/20  5/15/20  5/16/20  5/17/20  5/18/20  5/19/20  
3      596      604      615      617      624      628  

[1 rows x 121 columns]
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=564.574, BIC=568.953, Time=0.016 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constan

Fit ARIMA: (0, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=209.384, BIC=220.408, Time=0.088 seconds
Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=207.826, BIC=221.054, Time=0.070 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=205.689, BIC=212.303, Time=0.031 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=207.632, BIC=216.450, Time=0.130 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=209.518, BIC=220.542, Time=0.120 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=207.125, BIC=220.353, Time=0.218 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=209.812, BIC=225.245, Time=0.156 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=207.417, BIC=216.235, Time=0.080 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=209.416, BIC=220.440, Time=0.112 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=203.944, BIC=217.172, Time=0.608 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (const

...

Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=532.896, BIC=548.735, Time=0.267 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=529.021, BIC=542.597, Time=0.129 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=530.442, BIC=546.281, Time=0.192 seconds
Total fit time: 3.232 seconds
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
51        Vermont             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  5/10/20  5/11/20  5/12/20  5/13/20  \
51        0        0        0  ...       53       53       53       53   

    5/14/20  5/15/20  5/16/20  5/17/20  5/18/20  5/19/20  
51       53       53       53       54       54       54  

[1 rows x 121 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=185.219, BIC=189.441, Time=0.015 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=186.381, BIC=192.714, Time=0.055 seconds
Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (c

Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=492.680, BIC=501.377, Time=0.127 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=492.842, BIC=503.714, Time=0.172 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=492.628, BIC=505.675, Time=0.201 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=510.277, BIC=516.800, Time=0.055 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=492.837, BIC=501.535, Time=0.090 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=490.062, BIC=500.934, Time=0.517 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=494.365, BIC=507.411, Time=0.372 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=495.047, BIC=510.268, Time=0.360 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=502.559, BIC=511.257, Time=0.071 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=493.286, BIC=504.158, Time=0.132 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (const

Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=552.227, BIC=564.990, Time=0.381 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=554.243, BIC=569.133, Time=0.523 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=552.340, BIC=565.103, Time=0.251 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=554.225, BIC=569.115, Time=0.558 seconds
Total fit time: 5.510 seconds
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
55  West Virginia             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  5/10/20  5/11/20  5/12/20  5/13/20  \
55        0        0        0  ...       54       54       58       58   

    5/14/20  5/15/20  5/16/20  5/17/20  5/18/20  5/19/20  
55       60       64       64       67       68       68  

[1 rows x 121 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=188.495, BIC=192.319, Time=0.017 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (c

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=572.506, BIC=581.384, Time=0.194 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=574.440, BIC=585.538, Time=0.251 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=576.081, BIC=589.398, Time=0.420 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=578.063, BIC=593.600, Time=0.588 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=572.460, BIC=581.338, Time=0.133 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=574.426, BIC=585.524, Time=0.330 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=575.386, BIC=588.703, Time=0.468 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=578.269, BIC=593.805, Time=0.160 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=574.308, BIC=585.405, Time=0.171 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=576.101, BIC=589.418, Time=0.391 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (const

Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1300.977, BIC=1308.670, Time=0.133 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1297.023, BIC=1307.281, Time=0.221 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1298.965, BIC=1311.787, Time=0.278 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1300.735, BIC=1316.121, Time=0.374 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1316.468, BIC=1324.161, Time=0.030 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1297.695, BIC=1307.952, Time=0.227 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1298.836, BIC=1311.658, Time=0.365 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1298.342, BIC=1313.728, Time=0.403 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1300.325, BIC=1318.275, Time=0.587 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1302.354, BIC=1312.611, Time=0.128 seconds
Fit ARIMA: (2, 2, 1)